# Trials

In [ ]:
import pandas as pd

df = pd.read_csv("J44.9.csv")
df['data Internare'] = pd.to_datetime(df['data Internare'], format='%Y-%m-%d %H:%M:%S')

# Define the beginning of the year
start_of_year = pd.Timestamp('2013-01-01 00:00:00')

# Calculate the difference in minutes from the start of the year
df['month'] = (df['data Internare'] - start_of_year) / pd.Timedelta('30 days')


df.head(1000)

,identifier,Lat,Long,FO,data Internare,month
0,h5la19d0itxuihx,45.698868,21.433261,1023/2023,2023-01-13 21:00:39,122.162515
1,21er6sn98v351fd,45.704979,21.090164,1041/2023,2023-01-13 23:42:58,122.166272
2,osfjnqmtg8rswr2,45.696912,21.247647,1573/2023,2023-01-24 14:30:40,122.520154
3,9o66crbz29dxmm5,45.734998,21.203804,1856/2023,2023-01-27 22:32:47,122.631314
4,l6io00j9fydwtdv,45.726987,21.228252,957/2023,2023-01-13 08:58:26,122.145797
...,...,...,...,...,...,...
995,xo3h9iex3072a2p,45.720639,21.315646,15590/2018,2018-09-27 22:00:16,69.863895
996,o73l510jdtaubt0,45.597360,21.212316,14293/2018,2018-09-05 09:35:39,69.113325
997,ylumt54v3hhxauw,45.760974,21.206915,15278/2018,2018-09-24 08:37:07,69.745304
998,gkpyutfkc3x5yc5,45.965375,21.479750,14204/2018,2018-09-04 13:56:48,69.086037


Haversine

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

tdf = np.radians(df[['Lat', ' Long']])
tdf['month'] = df['month']

# Define the epsilon parameter (maximum distance between two samples for them to be considered as in the same neighborhood)
# Assuming we want an eps of 10 kilometers, we convert this to radians.
# Earth's radius in kilometers (mean radius = 6,371 km)
earth_radius_km = 6371.0
eps_km = 3
eps = eps_km / earth_radius_km

# Perform DBSCAN clustering with Haversine distance
db = DBSCAN(eps=eps, min_samples=2, metric='haversine').fit(tdf[['Lat', ' Long']])

# Get cluster labels
labels = db.labels_

# Add labels to the DataFrame
df['label'] = labels

# Separate noise points (label = -1)
noise_df = df[df['label'] == -1]
cluster_df = df[df['label'] != -1]

# Plot the clusters using Plotly
fig = px.scatter_mapbox(
    cluster_df,
    lat='Lat',
    lon=' Long',
    color='label',
    title='DBSCAN Clusters with Haversine Distance',
    color_continuous_scale=px.colors.sequential.Viridis,
    size_max=15,
    zoom=3
)

# Add noise points to the plot with a different color (e.g., black)
fig.add_scattermapbox(
    lat=noise_df['Lat'],
    lon=noise_df[' Long'],
    mode='markers',
    marker=dict(size=9, color='red'),
    name='Noise'
)

fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7.5,
                  mapbox_center={"lat": 45.7407, "lon": 21.2196},
                  margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

Euclidian?

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

#tdf = np.radians(df[['Lat', ' Long']])
tdf['month'] = df['month']
tdf['Lat'] = df['Lat']
tdf[' Long'] = df[' Long']

# Define the epsilon parameter (maximum distance between two samples for them to be considered as in the same neighborhood)
# Assuming we want an eps of 10 kilometers, we convert this to radians.
# Earth's radius in kilometers (mean radius = 6,371 km)

earth_radius_km = 6371.0
eps_km = 3
eps = eps_km / earth_radius_km

# Perform DBSCAN clustering with Haversine distance
db = DBSCAN(eps=3, min_samples=3).fit(df[['Lat', ' Long','month']])

# Get cluster labels
labels = db.labels_

# Add labels to the DataFrame
df['label'] = labels

# Separate noise points (label = -1)
noise_df = df[df['label'] == -1]
cluster_df = df[df['label'] != -1]

# Plot the clusters using Plotly
fig = px.scatter_mapbox(
    cluster_df,
    lat='Lat',
    lon=' Long',
    color='label',
    title='DBSCAN Clusters with Haversine Distance',
    color_continuous_scale=px.colors.sequential.Viridis,
    size_max=15,
    zoom=3
)

# Add noise points to the plot with a different color (e.g., black)
fig.add_scattermapbox(
    lat=noise_df['Lat'],
    lon=noise_df[' Long'],
    mode='markers',
    marker=dict(size=9, color='red'),
    name='Noise'
)

fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7.5,
                  mapbox_center={"lat": 45.7407, "lon": 21.2196},
                  margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

OPTICS

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from sklearn.cluster import OPTICS

tdf = np.radians(df[['Lat', ' Long']])


# Define the epsilon parameter (maximum distance between two samples for them to be considered as in the same neighborhood)
# Assuming we want an eps of 10 kilometers, we convert this to radians.
# Earth's radius in kilometers (mean radius = 6,371 km)
earth_radius_km = 6371.0
eps_km = 3
eps = eps_km / earth_radius_km

# Fit OPTICS
optics = OPTICS(min_samples=5, xi=0.05, min_cluster_size=0.0005, metric='haversine')
optics.fit(tdf)

# Get cluster labels
labels = optics.labels_

# Add labels to the DataFrame
df['label'] = labels

# Separate noise points (label = -1)
noise_df = df[df['label'] == -1]
cluster_df = df[df['label'] != -1]

# Plot the clusters using Plotly
fig = px.scatter_mapbox(
    cluster_df,
    lat='Lat',
    lon=' Long',
    color='label',
    title='DBSCAN Clusters with Haversine Distance',
    color_continuous_scale=px.colors.sequential.Viridis,
    size_max=15,
    zoom=3
)

# Add noise points to the plot with a different color (e.g., black)
fig.add_scattermapbox(
    lat=noise_df['Lat'],
    lon=noise_df[' Long'],
    mode='markers',
    marker=dict(size=9, color='red'),
    name='Noise'
)

fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7.5,
                  mapbox_center={"lat": 45.7407, "lon": 21.2196},
                  margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:995: RuntimeWarning:

divide by zero encountered in divide



In [ ]:
tdf

,Lat,Long
0,0.797596,0.374081
1,0.797702,0.368093
2,0.797562,0.370841
3,0.798226,0.370076
4,0.798086,0.370503
...,...,...
1956,0.799338,0.370836
1957,0.797525,0.370595
1958,0.798785,0.370460
1959,0.798251,0.370178


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from sklearn.cluster import OPTICS
from sklearn.metrics.pairwise import haversine_distances
from sklearn.metrics import pairwise_distances
import plotly.graph_objects as go

tdf = np.radians(df[['Lat', ' Long']])
df_time = df['month']
tdf = pd.concat([tdf,df_time], axis = 1)


# Define the epsilon parameter (maximum distance between two samples for them to be considered as in the same neighborhood)
# Assuming we want an eps of 10 kilometers, we convert this to radians.
# Earth's radius in kilometers (mean radius = 6,371 km)
earth_radius_km = 6371.0
eps_km = 3
eps = eps_km / earth_radius_km

latitude_std = np.std(df['Lat'])
longitude_std = np.std(df[' Long'])
time_std = np.std(df['month'])

# Inverse of standard deviations (or range) as weights
spatial_weight = 1 / (latitude_std + longitude_std)
temporal_weight = 1 / time_std

# Normalize weights to ensure they are comparable
#total_weight = spatial_weight + temporal_weight
#spatial_weight /= total_weight
#temporal_weight /= total_weight

spatial_weight *= 0.5
temporal_weight *= 0.5

#def custom_distance(p1, p2):
     # Compute Haversine distance for spatial components
    #spatial_distance = haversine_distances([p1[:2]], [p2[:2]])[0][0]

    # Compute temporal distance
    #temporal_distance = abs(p1[2] - p2[2])

    # Combine distances with respective weights
    #return spatial_weight * spatial_distance + temporal_weight * temporal_distance

    # Compute weighted distances
    #spatial_distance = euclidean(spatial1, spatial2)
    #temporal_distance = abs(temporal1 - temporal2)

    # Combine with weights
    #return spatial_weight * spatial_distance + temporal_weight * temporal_distance

distance_matrix_time = pairwise_distances(tdf, metric=lambda x, y: abs(x[2] - y[2]))
df_spatial = tdf[['Lat',' Long']]
distance_matrix_haversine = pairwise_distances(df_spatial, metric='haversine')
distance_matrix = distance_matrix_time * temporal_weight + distance_matrix_haversine * spatial_weight

# Apply OPTICS with precomputed distance matrix
optics = OPTICS(min_samples=5, xi=0.05, min_cluster_size=0.0005, metric='precomputed')
optics.fit(distance_matrix)

# Fit OPTICS
#optics = OPTICS(min_samples=5, xi=0.05, min_cluster_size=0.0005, metric='haversine')
#optics.fit(tdf)

# Get cluster labels
labels = optics.labels_

# Add labels to the DataFrame
df['label'] = labels

df = df.sort_values(by='month')

# Separate noise points (label = -1)
noise_df = df[df['label'] == -1]
cluster_df = df[df['label'] != -1]

# Plot the clusters using Plotly
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
        lat=noise_df['Lat'],
        lon=noise_df[' Long'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=5,
            color='rgb(0, 0, 0)',
            opacity=0.7
        ),
        text=cluster_df['data Internare'].dt.strftime('%Y-%m-%d %H:%M') + '<br>Noise',
    ))

fig.add_trace(go.Scattermapbox(
        lat=cluster_df['Lat'],
        lon=cluster_df[' Long'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color=cluster_df['label'], colorscale='mygbm',
            opacity=0.7
        ),
        text=cluster_df['data Internare'].dt.strftime('%Y-%m-%d %H:%M') + '<br>' + cluster_df['label'].astype(str),
    ))

fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7.5,
                  mapbox_center={"lat": 45.7407, "lon": 21.2196},
                  margin={"r":0,"t":0,"l":0,"b":0})


fig.show()

In [ ]:
fig = px.scatter_mapbox(
    cluster_df,
    lat='Lat',
    lon=' Long',
    color='label',
    title='DBSCAN Clusters with Haversine Distance',
    animation_frame='label',
    animation_group='label',
    size_max=15,
    zoom=3,
)

fig.update_traces(marker=dict(size=10,
            color=cluster_df['label'], colorscale='mygbm',
            opacity=0.7),
            )

fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=7.5,
                  mapbox_center={"lat": 45.7407, "lon": 21.2196},
                  margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

# ASTM

# Functions

In [ ]:
from sklearn.metrics import silhouette_score
from itertools import product

def read_file(file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Convert the 'data Internare' column to datetime format
    df['data Internare'] = pd.to_datetime(df['data Internare'], format='%Y-%m-%d %H:%M:%S')

    # Define the beginning of the year
    start_of_year = pd.Timestamp('2013-01-01 00:00:00')

    # Rename the 'Lng' column to 'Long'
    df.rename(columns={'Lng': ' Long'}, inplace=True)

    # Calculate the difference in months from the start of the year
    df['month'] = (df['data Internare'] - start_of_year) / pd.Timedelta('30 days')

    return df

def apply_optics_clustering(df, spatial_weight, temporal_weight, eps_km=3, min_samples=5, xi=0.05, min_cluster_size=0.0005):
    # Convert latitude and longitude to radians
    tdf = np.radians(df[['Lat', ' Long']])
    df_time = df['month']
    tdf = pd.concat([tdf, df_time], axis=1)

    # Earth's radius in kilometers
    earth_radius_km = 6371.0
    eps = eps_km / earth_radius_km

    # Calculate distance matrices
    distance_matrix_time = pairwise_distances(tdf, metric=lambda x, y: abs(x[2] - y[2]))
    df_spatial = tdf[['Lat', ' Long']]
    distance_matrix_haversine = pairwise_distances(df_spatial, metric='haversine')

    # Calculate weighted distance matrix
    distance_matrix = distance_matrix_time * temporal_weight + distance_matrix_haversine * spatial_weight

    # Apply OPTICS with precomputed distance matrix
    optics = OPTICS(min_samples=min_samples, xi=xi, min_cluster_size=min_cluster_size, metric='precomputed')
    optics.fit(distance_matrix)

    # Get cluster labels
    labels = optics.labels_

    # Add labels to the DataFrame
    df['label'] = labels
    df = df.sort_values(by='month')

    return df, distance_matrix

def plot_clusters(df):
    # Separate noise points (label = -1) and clusters
    noise_df = df[df['label'] == -1]
    cluster_df = df[df['label'] != -1]

    # Plot the clusters using Plotly
    fig = go.Figure()

    fig.add_trace(go.Scattermapbox(
        lat=noise_df['Lat'],
        lon=noise_df[' Long'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=5,
            color='rgb(0, 0, 0)',
            opacity=0.7
        ),
        text=noise_df['data Internare'].dt.strftime('%Y-%m-%d %H:%M') + '<br>Noise',
    ))

    fig.add_trace(go.Scattermapbox(
        lat=cluster_df['Lat'],
        lon=cluster_df[' Long'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color=cluster_df['label'],
            colorscale='mygbm',
            opacity=0.7
        ),
        text=cluster_df['data Internare'].dt.strftime('%Y-%m-%d %H:%M') + '<br>' + cluster_df['label'].astype(str),
    ))

    fig.update_layout(mapbox_style="carto-positron",
                      mapbox_zoom=7.5,
                      mapbox_center={"lat": 45.7407, "lon": 21.2196},
                      margin={"r": 0, "t": 0, "l": 0, "b": 0})

    fig.show()

def plot_cluster_slices(df, title='DBSCAN Clusters with Haversine Distance', zoom=7.5, center_lat=45.7407, center_lon=21.2196):

    cluster_df = df[df['label'] != -1]

    # Create the scatter mapbox plot
    fig = px.scatter_mapbox(
        cluster_df,
        lat='Lat',
        lon=' Long',  # Ensure 'Long' is used consistently
        color='label',
        title=title,
        animation_frame='label',
        animation_group='label',
        size_max=15,
        zoom=zoom,
    )

    # Update traces with specific marker settings
    fig.update_traces(marker=dict(
        size=10,
        color=cluster_df['label'],
        colorscale='mygbm',
        opacity=0.7
    ))

    # Update layout with mapbox style and center coordinates
    fig.update_layout(mapbox_style="carto-positron",
                      mapbox_zoom=zoom,
                      mapbox_center={"lat": center_lat, "lon": center_lon},
                      margin={"r": 0, "t": 0, "l": 0, "b": 0})

    # Display the plot
    fig.show()


    # Function to evaluate the clustering
def evaluate_clustering(labels, distance_matrix):
    if len(set(labels)) > 1:  # Ensure there is more than one cluster
        return silhouette_score(distance_matrix, labels, metric="precomputed")
    else:
        return -1  # Return -1 for cases where clustering is not meaningful

# Function to perform grid search for optimal spatial and temporal weights
def grid_search_optimal_weights(df, spatial_weights, eps_km=3, min_samples=10, xi=0.05, min_cluster_size=0.0005):
    best_score = -1
    best_spatial_weight = None
    best_temporal_weight = None

    for spatial_weight in spatial_weights:
        # Apply the clustering function
        temporal_weight = 1.0 - spatial_weight
        clustered_df, distance_matrix = apply_optics_clustering(df.copy(), spatial_weight, temporal_weight, eps_km, min_samples, xi, min_cluster_size)

        # Evaluate the clustering
        score = evaluate_clustering(clustered_df['label'], distance_matrix)

        print(f"Spatial Weight: {spatial_weight}, Temporal Weight: {temporal_weight}, Score: {score}")

        # Check if this is the best score
        if score > best_score:
            best_score = score
            best_spatial_weight = spatial_weight
            best_temporal_weight = temporal_weight

    print(f"\nBest Spatial Weight: {best_spatial_weight}")
    print(f"Best Temporal Weight: {best_temporal_weight}")
    print(f"Best Silhouette Score: {best_score}")

    return best_spatial_weight, best_temporal_weight


# J45

In [ ]:
j45_df = read_file('J45-modified.csv')
j45_df.head()

apply_optics_clustering(j45_df, 0.5, 0.5)

KeyboardInterrupt: 

In [ ]:
plot_cluster_slices(j45_df)

# J45.0

In [ ]:
j450_df = read_file('J45.0-modified.csv')
j450_df.head()

spatial_weights = np.linspace(0.1, 0.9, 9)  # Define the range for spatial weights
 # Define the range for temporal weights
best_spatial_weight, best_temporal_weight = grid_search_optimal_weights(j450_df, spatial_weights)
apply_optics_clustering(j450_df, best_spatial_weight, best_temporal_weight)

Spatial Weight: 0.1, Temporal Weight: 0.9, Score: -0.17675468872280933
Spatial Weight: 0.2, Temporal Weight: 0.8, Score: -0.17721723371468565
Spatial Weight: 0.30000000000000004, Temporal Weight: 0.7, Score: -0.18638414663445213
Spatial Weight: 0.4, Temporal Weight: 0.6, Score: -0.18415044826194651
Spatial Weight: 0.5, Temporal Weight: 0.5, Score: -0.18020704630756504
Spatial Weight: 0.6, Temporal Weight: 0.4, Score: -0.1842201463413697
Spatial Weight: 0.7000000000000001, Temporal Weight: 0.29999999999999993, Score: -0.18800000106679748
Spatial Weight: 0.8, Temporal Weight: 0.19999999999999996, Score: -0.19300096538227426
Spatial Weight: 0.9, Temporal Weight: 0.09999999999999998, Score: -0.20828582385621344

Best Spatial Weight: 0.1
Best Temporal Weight: 0.9
Best Silhouette Score: -0.17675468872280933


(           identifier          FO      data Internare        Lat       Long  \
 4930  xovoed7r86so3hg     72/2013 2013-01-04 11:08:39  45.719899  21.157768   
 4926  bwjm4pdcjaavwyo    125/2013 2013-01-07 09:53:15  45.969180  20.770985   
 4931  igaxx5z05gv0cem    173/2013 2013-01-08 08:54:07  45.837414  21.096789   
 4927  rpc34rc1hv38pqz    227/2013 2013-01-09 08:59:24  45.745880  21.198654   
 4928  wql88k4c7ffxam3    363/2013 2013-01-11 15:00:44  45.748872  21.208679   
 ...               ...         ...                 ...        ...        ...   
 627   4620toh5nz1u36v  22197/2023 2023-11-28 09:06:02  45.776878  21.220126   
 628   wtqnlets9cg3ojb  22201/2023 2023-11-28 09:12:16  45.546514  20.990175   
 629   qmima1gc07mer17  22205/2023 2023-11-28 09:17:26  45.731071  21.204905   
 630   liecg0a141ygz0i  22228/2023 2023-11-28 10:18:01  45.762213  21.024143   
 631   to14r14gai352is  22239/2023 2023-11-28 10:40:11  45.734127  21.237930   
 
            month  label  
 4930    0.

In [ ]:
plot_clusters(j450_df)

In [ ]:
plot_cluster_slices(j450_df)

# J45.1

In [ ]:
j451_df = read_file('J45.1-modified.csv')
j451_df.head()

apply_optics_clustering(j451_df, 0.5, 0.5)

In [ ]:
plot_cluster_slices(j451_df)

# J45.9

In [ ]:
j459_df = read_file('J45.9-modified.csv')
j459_df.head()

spatial_weights = np.linspace(0.1, 0.9, 9)  # Define the range for spatial weights
 # Define the range for temporal weights
best_spatial_weight, best_temporal_weight = grid_search_optimal_weights(j459_df, spatial_weights)
apply_optics_clustering(j459_df, best_spatial_weight, best_temporal_weight)

In [ ]:
plot_cluster_slices(j459_df)

# J46

In [ ]:
j46_df = read_file('J46-modified.csv')
j46_df.head()

apply_optics_clustering(j46_df, 0.5, 0.5)

In [ ]:
plot_cluster_slices(j46_df)

## BPOC

# J44.0

In [ ]:
j440_df = read_file('J44.0.csv')
j440_df.head()

apply_optics_clustering(j440_df, 0.5, 0.5)

In [ ]:
plot_cluster_slices(j440_df)

# J44.1

In [ ]:
j441_df = read_file('J44.1.csv')
j441_df.head()

apply_optics_clustering(j441_df, 0.5, 0.5)

In [ ]:
plot_cluster_slices(j441_df)